Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hidden_unit = 1024
reg = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_unit]))
    b1 = tf.Variable(tf.zeros([hidden_unit]))
    w2 = tf.Variable(
        tf.truncated_normal([hidden_unit, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    activations = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    logits = tf.matmul(activations, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits = logits)) \
        + (tf.nn.l2_loss(w2) + tf.nn.l2_loss(w1)) * reg
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_activations = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_logits = tf.matmul(valid_activations, w2) + b2
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_activations = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_logits = tf.matmul(test_activations, w2) + b2
    test_prediction = tf.nn.softmax(test_logits)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset : offset + batch_size, :]
        batch_labels = train_labels[offset : offset + batch_size, :]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f", (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
    

Initialized
Minibatch loss at step %d: %f (0, 3436.7004)
Minibatch accuracy: 16.4%
Validation accuracy: 23.7%
Minibatch loss at step %d: %f (500, 21.42291)
Minibatch accuracy: 80.5%
Validation accuracy: 84.5%
Minibatch loss at step %d: %f (1000, 0.9729315)
Minibatch accuracy: 76.6%
Validation accuracy: 83.7%
Minibatch loss at step %d: %f (1500, 0.72054315)
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step %d: %f (2000, 0.8153635)
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Minibatch loss at step %d: %f (2500, 0.67551816)
Minibatch accuracy: 85.9%
Validation accuracy: 83.9%
Minibatch loss at step %d: %f (3000, 0.7086573)
Minibatch accuracy: 82.8%
Validation accuracy: 83.7%
Test accuracy: 89.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = batch_size * (step % 10)
        batch_data = train_dataset[offset : offset + batch_size, :]
        batch_labels = train_labels[offset : offset + batch_size, :]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f", (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step %d: %f (0, 3492.1624)
Minibatch accuracy: 8.6%
Validation accuracy: 28.9%
Minibatch loss at step %d: %f (500, 21.143078)
Minibatch accuracy: 99.2%
Validation accuracy: 80.6%
Minibatch loss at step %d: %f (1000, 0.6996076)
Minibatch accuracy: 96.9%
Validation accuracy: 80.5%
Minibatch loss at step %d: %f (1500, 0.5203917)
Minibatch accuracy: 96.9%
Validation accuracy: 80.4%
Minibatch loss at step %d: %f (2000, 0.50055814)
Minibatch accuracy: 96.9%
Validation accuracy: 80.3%
Minibatch loss at step %d: %f (2500, 0.4903527)
Minibatch accuracy: 96.9%
Validation accuracy: 80.3%
Minibatch loss at step %d: %f (3000, 0.48449227)
Minibatch accuracy: 96.9%
Validation accuracy: 80.2%
Minibatch loss at step %d: %f (3500, 0.48043552)
Minibatch accuracy: 96.9%
Validation accuracy: 80.2%
Minibatch loss at step %d: %f (4000, 0.47743618)
Minibatch accuracy: 96.9%
Validation accuracy: 80.0%
Minibatch loss at step %d: %f (4500, 0.47530296)
Minibatch accuracy: 96.9%
Valid

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128
hidden_unit = 1024
reg = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_unit]))
    b1 = tf.Variable(tf.zeros([hidden_unit]))
    w2 = tf.Variable(
        tf.truncated_normal([hidden_unit, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    activations = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), 0.3)
    logits = tf.matmul(activations, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits = logits)) \
        + (tf.nn.l2_loss(w2) + tf.nn.l2_loss(w1)) * reg
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_activations = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_logits = tf.matmul(valid_activations, w2) + b2
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_activations = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_logits = tf.matmul(test_activations, w2) + b2
    test_prediction = tf.nn.softmax(test_logits)

In [9]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = batch_size * (step % 10)
        batch_data = train_dataset[offset : offset + batch_size, :]
        batch_labels = train_labels[offset : offset + batch_size, :]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f", (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step %d: %f (0, 3714.848)
Minibatch accuracy: 13.3%
Validation accuracy: 33.7%
Minibatch loss at step %d: %f (500, 22.051369)
Minibatch accuracy: 98.4%
Validation accuracy: 81.3%
Minibatch loss at step %d: %f (1000, 0.9123007)
Minibatch accuracy: 93.8%
Validation accuracy: 80.9%
Minibatch loss at step %d: %f (1500, 0.69488263)
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%
Minibatch loss at step %d: %f (2000, 0.6989187)
Minibatch accuracy: 91.4%
Validation accuracy: 80.4%
Minibatch loss at step %d: %f (2500, 0.66875744)
Minibatch accuracy: 94.5%
Validation accuracy: 80.9%
Minibatch loss at step %d: %f (3000, 0.64036053)
Minibatch accuracy: 95.3%
Validation accuracy: 80.8%
Minibatch loss at step %d: %f (3500, 0.66400784)
Minibatch accuracy: 91.4%
Validation accuracy: 80.5%
Minibatch loss at step %d: %f (4000, 0.6442788)
Minibatch accuracy: 93.0%
Validation accuracy: 80.8%
Minibatch loss at step %d: %f (4500, 0.6585959)
Minibatch accuracy: 93.0%
Valida

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
# OPTION 1: use multiple layers

batch_size = 128
h1_unit = 512
h2_unit = 512
reg = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w1 = tf.Variable(
        0.5 * tf.truncated_normal([image_size * image_size, h1_unit]))
    b1 = tf.Variable(tf.zeros([h1_unit]))
    w2 = tf.Variable(
        0.5 * tf.truncated_normal([h1_unit, h2_unit]))
    b2 = tf.Variable(tf.zeros([h2_unit]))
    w3 = tf.Variable(
        0.5 * tf.truncated_normal([h2_unit, num_labels]))
    b3 = tf.Variable(tf.zeros([num_labels]))
    
    a1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    a2 = tf.nn.relu(tf.matmul(a1, w2) + b2)
    logits = tf.matmul(a2, w3) + b3
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits = logits)) \
        + (tf.nn.l2_loss(w2) + tf.nn.l2_loss(w1) + tf.nn.l2_loss(w3)) * reg
    
#     optimizer = tf.train.GradientDescentOptimizer(0.005).minimize(loss)
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_a1 = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_a2 = tf.nn.relu(tf.matmul(valid_a1, w2) + b2)
    valid_logits = tf.matmul(valid_a2, w3) + b3
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_a1 = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_a2 = tf.nn.relu(tf.matmul(test_a1, w2) + b2)
    test_logits = tf.matmul(test_a2, w3) + b3
    test_prediction = tf.nn.softmax(test_logits)

In [16]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset : offset + batch_size, :]
        batch_labels = train_labels[offset : offset + batch_size, :]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" %(step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1035.076050
Minibatch accuracy: 13.3%
Validation accuracy: 10.5%
Minibatch loss at step 500: 574.204590
Minibatch accuracy: 73.4%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 438.236450
Minibatch accuracy: 78.1%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 328.630554
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 233.525757
Minibatch accuracy: 82.0%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 156.475250
Minibatch accuracy: 86.7%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 98.120529
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 56.269176
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Minibatch loss at step 4000: 29.678440
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Minibatch loss at step 4500: 14.378203
Minibatch accuracy: 90.6%
Validation accuracy: 86.6%
Minibatch loss at step 5000: 6.877000
Minibatch accuracy: 85.9%
V

In [12]:
# OPTION 2: use learning rate decay

batch_size = 128
hidden_unit = 1024
reg = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                     shape = (batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_unit]))
    b1 = tf.Variable(tf.zeros([hidden_unit]))
    w2 = tf.Variable(
        tf.truncated_normal([hidden_unit, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    activations = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    logits = tf.matmul(activations, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits = logits)) \
        + (tf.nn.l2_loss(w2) + tf.nn.l2_loss(w1)) * reg
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps = 100, decay_rate=0.9)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_activations = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_logits = tf.matmul(valid_activations, w2) + b2
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_activations = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_logits = tf.matmul(test_activations, w2) + b2
    test_prediction = tf.nn.softmax(test_logits)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset : offset + batch_size, :]
        batch_labels = train_labels[offset : offset + batch_size, :]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f", (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step %d: %f (0, 3473.8503)
Minibatch accuracy: 9.4%
Validation accuracy: 35.3%
Minibatch loss at step %d: %f (500, 63.567585)
Minibatch accuracy: 81.2%
Validation accuracy: 85.0%
Minibatch loss at step %d: %f (1000, 7.117882)
Minibatch accuracy: 78.9%
Validation accuracy: 84.6%
Minibatch loss at step %d: %f (1500, 2.3636181)
Minibatch accuracy: 82.0%
Validation accuracy: 84.9%
Minibatch loss at step %d: %f (2000, 1.5296755)
Minibatch accuracy: 81.2%
Validation accuracy: 84.8%
Minibatch loss at step %d: %f (2500, 1.1277004)
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step %d: %f (3000, 1.0433185)
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Test accuracy: 90.5%
